In [1]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv() 

#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-1.5-flash-latest",google_api_key=os.environ["GOOGLE_API_KEY"])

In [2]:
print(llm("10 lines about abdul kalam and dr.jaisankar"))

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Abdul Kalam, a visionary, soared with fire,
A missile man, fulfilling India's desire.
Dr. Jaishankar, in diplomacy's art,
Navigates global currents, playing a smart part.

Kalam's legacy, in science's bright gleam,
Inspires youth to chase a celestial dream.
Jaishankar's wisdom, in strategic design,
Shapes India's standing, a powerful sign.

Two giants distinct, yet both serve the nation,
With dedication, skill, and deep devotion.



#### Connect with database and ask some basic questions

In [3]:

#from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities import SQLDatabase


In [5]:
db_user = "root"
db_password = "sr2910"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [6]:
#Convert question to SQL query
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there"})
response

'Question: How many customers are there\nSQLQuery: SELECT COUNT(DISTINCT `CustomerID`) AS `Number of Customers` FROM `sales_tb`'

In [7]:
cleaned_query = response.split("SQLQuery:")[-1].strip()
print(cleaned_query)

SELECT COUNT(DISTINCT `CustomerID`) AS `Number of Customers` FROM `sales_tb`


In [8]:
# Execute the cleaned query
result = db.run(cleaned_query)
print(result)

[(29,)]


In [9]:
chain = create_sql_query_chain(llm, db)
def execute_query(question):
    try:
        # Generate SQL query from question
        response = chain.invoke({"question": question})
        print(response)
        print("###################################################")
        # Strip the formatting markers from the response
        cleaned_query = response.split("SQLQuery:")[-1].strip()
        print(cleaned_query)
        print("###################################################")        
        # Execute the cleaned query
        result = db.run(cleaned_query)
        print("###################################################")        
        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")


In [10]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

Question: How many unique customers are there for each product category
SQLQuery: SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomerCount` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomerCount` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
###################################################
[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [17]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)

Question: Calculate total sales amount per product category:
SQLQuery: SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
###################################################
[('Beauty', Decimal('1455.00')), ('Clothing', Decimal('5040.00')), ('Electronics', Decimal('5310.00'))]


In [18]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)

Question: calculates the average age of customers grouped by gender.
SQLQuery: SELECT `Gender`, AVG(`Age`) AS `AverageAge` FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
###################################################
SELECT `Gender`, AVG(`Age`) AS `AverageAge` FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
###################################################
###################################################
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]


In [19]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

Question: identify the top spending customers based on their total amount spent.
SQLQuery: SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent` FROM `sales_tb` GROUP BY `CustomerID` ORDER BY `TotalSpent` DESC LIMIT 5;
###################################################
SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent` FROM `sales_tb` GROUP BY `CustomerID` ORDER BY `TotalSpent` DESC LIMIT 5;
###################################################
###################################################
[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]


In [20]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

Question: counts the number of transactions made each month.
SQLQuery: SELECT MONTH(`Date`) AS `Month`, COUNT(*) AS `TransactionCount` FROM `sales_tb` GROUP BY `Month` LIMIT 5;
###################################################
SELECT MONTH(`Date`) AS `Month`, COUNT(*) AS `TransactionCount` FROM `sales_tb` GROUP BY `Month` LIMIT 5;
###################################################
###################################################
[(11, 3), (2, 4), (1, 4), (5, 2), (4, 5)]


In [21]:
q6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(q6)

Question: calculates the total sales amount and average price per unit for each product category.
SQLQuery: SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales`, AVG(`PriceperUnit`) AS `AveragePricePerUnit` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales`, AVG(`PriceperUnit`) AS `AveragePricePerUnit` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
###################################################
[('Beauty', Decimal('1455.00'), Decimal('153.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Electronics', Decimal('5310.00'), Decimal('238.750000'))]
